# Tasca mètodes de mostreig
**Descripció**

Aprèn a realitzar mostreig de les dades amb Python.


In [1]:
import numpy as np
import pandas as pd

import random as rd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE



## Nivell 1

- Exercici 1

Agafa un conjunt de dades de tema esportiu que t'agradi. Realitza un mostreig de les dades generant una mostra aleatòria simple i una mostra sistemàtica.


In [2]:
players=pd.read_table('./datasets/mlb.txt')

In [3]:
players.tail()

,player,team,position,salary
823,Jordan Zimmerman,Washington Nationals,Pitcher,401.0
824,Ian Desmond,Washington Nationals,Shortstop,400.0
825,Ross Detwiler,Washington Nationals,Pitcher,400.0
826,Jesse English,Washington Nationals,Pitcher,400.0
827,Willy Taveras,Washington Nationals,Outfielder,400.0


In [4]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828 entries, 0 to 827
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   player    828 non-null    object 
 1   team      828 non-null    object 
 2   position  828 non-null    object 
 3   salary    828 non-null    float64
dtypes: float64(1), object(3)
memory usage: 26.0+ KB


**Muestreo aleatorio simple**

In [5]:
# Función sample de Pandas - 10%

muestra_aleatoria=players.sample(frac=0.1,random_state=1984,)
muestra_aleatoria.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83 entries, 801 to 809
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   player    83 non-null     object 
 1   team      83 non-null     object 
 2   position  83 non-null     object 
 3   salary    83 non-null     float64
dtypes: float64(1), object(3)
memory usage: 3.2+ KB


In [6]:
# % Diferencia de medias
abs(muestra_aleatoria.salary.mean()-players.salary.mean())/muestra_aleatoria.salary.mean()*100

1.568620914385014

In [7]:
muestra_aleatoria.position.value_counts(normalize=True)

Pitcher           0.469880
Outfielder        0.132530
Second Baseman    0.096386
Catcher           0.096386
First Baseman     0.084337
Shortstop         0.072289
Third Baseman     0.048193
Name: position, dtype: float64

In [8]:
players.position.value_counts(normalize=True)

Pitcher              0.495169
Outfielder           0.178744
Catcher              0.080918
Shortstop            0.067633
Second Baseman       0.057971
Third Baseman        0.055556
First Baseman        0.050725
Designated Hitter    0.008454
Infielder            0.004831
Name: position, dtype: float64

**Muestreo sistemático**

Para el muestreo sistemático, desarrollo el siguiente proceso: 
1. Escojo un número aleatoriamente entre todos los índices (0-827).
2. Calculo la longitud entre muestras (intervalo).
3. Construyo dos listas, una con números mayores del numero aleatorio y otra con numeros menores.
3. Genero lista de numeros mayores del numero aleatorio. Tomando muestras cada 10 elementos desde el némero aleatorio.
4. Para la lista de números inferiores, el índice de inicio lo obtengo del resto del último intervalo y de nuevo voy tomando muestras cada 10 elementos hasta el número aleatorio.  

In [9]:
# Primer número aleatorio con función randint
rd.seed(1984)
inicio=rd.randint(0,len(players))
print('Primer número aleatorio: ',inicio)

# Intervalo muestreo para muestra 10%  
intervalo=int(1/0.1)
print('Intervalo de muestreo: ',intervalo)

# Muestras desde el primer número aleatorio con intervalo fijo.
muestra_sistematica_hasta_final=[i for i in range(inicio,len(players),10)]

# Valor del intervalo menos elementos desde último indice.
inicio_segunda=intervalo-((len(players))-muestra_sistematica_hasta_final[-1])
# Muestras desde inicio del array numero aleatorio.
muestra_sistematica_desde_inicio=[i for i in range(inicio_segunda,inicio,10)]
# Unión de muestras
muestra_sistematica_lista=muestra_sistematica_desde_inicio+muestra_sistematica_hasta_final
print('Número de muestras: ',len(muestra_sistematica_lista))

Primer número aleatorio:  372
Intervalo de muestreo:  10
Número de muestras:  83


In [10]:
muestra_sistematica=players.iloc[muestra_sistematica_lista]

In [11]:
abs(muestra_sistematica.salary.mean()-players.salary.mean())/muestra_sistematica.salary.mean()*100

2.0522257999025983

In [12]:
muestra_sistematica.position.value_counts(normalize=True)

Pitcher           0.506024
Outfielder        0.120482
Second Baseman    0.096386
Third Baseman     0.072289
Catcher           0.072289
First Baseman     0.072289
Shortstop         0.060241
Name: position, dtype: float64



## Nivell 2

- Exercici 2

Continua amb el conjunt de dades de tema esportiu i genera una mostra estratificada i una mostra utilitzant SMOTE (Synthetic Minority Oversampling Technique).


Voy a hacer un muestreo estratificado por posición de los jugadores. Para ello, agrupo por position y con apply y una función lambda hago un muestreo de cada grupo. 

In [13]:
muestra_estratificada_pd=(
    
    players.groupby('position').apply(lambda x: x.sample(frac=0.1,
                                                         random_state=1984)
                                     ).reset_index(level=0,drop=True)
    )

In [14]:
abs(muestra_estratificada_pd.salary.mean()-players.salary.mean())/muestra_estratificada_pd.salary.mean()*100

6.0125159184621895

In [15]:
muestra_estratificada_pd.position.value_counts(normalize=True)

Pitcher              0.488095
Outfielder           0.178571
Catcher              0.083333
Shortstop            0.071429
Third Baseman        0.059524
Second Baseman       0.059524
First Baseman        0.047619
Designated Hitter    0.011905
Name: position, dtype: float64

Aunque también podemos utilizar el módulo models_selection de Scikit Learn donde hay alguna clase para muestreo estratificado.

In [16]:
train, muestra_estratificada_SL=train_test_split(players,
                                                test_size=0.1, 
                                                random_state=1984,
                                                stratify=players.position)

In [17]:
abs(muestra_estratificada_SL.salary.mean()-players.salary.mean())/muestra_estratificada_SL.salary.mean()*100

15.636310723417132

In [18]:
muestra_estratificada_SL.position.value_counts(normalize=True)#players.position.value_counts(normalize=True)

Pitcher              0.493976
Outfielder           0.180723
Catcher              0.084337
Shortstop            0.072289
Second Baseman       0.060241
Third Baseman        0.048193
First Baseman        0.048193
Designated Hitter    0.012048
Name: position, dtype: float64

Como podemos observar tenemos clases muy pequeñas puede ocurrir que en el muestreo estratificadas no aparezcan esas clases en la muestra. 

**SMOTE**

In [19]:
oversample=SMOTE(sampling_strategy='not majority',
                random_state=1984,
                k_neighbors=3)

In [20]:
players_overs, position_overs=oversample.fit_resample(players[['salary']],players[['position']])

In [21]:
position_overs.value_counts()

position         
Catcher              410
Designated Hitter    410
First Baseman        410
Infielder            410
Outfielder           410
Pitcher              410
Second Baseman       410
Shortstop            410
Third Baseman        410
dtype: int64

In [22]:
players_overs['position']=position_overs

In [23]:
muestra_estratificada_SMOTE=(
    
    players_overs.groupby('position').apply(lambda x: x.sample(n=83,
                                                               random_state=1984)
                                                                 ).reset_index(level=0,drop=True)
    )

In [24]:
abs(muestra_estratificada_SMOTE.salary.mean()-players.salary.mean())/muestra_estratificada_SMOTE.salary.mean()*100

2.103492172458711

In [25]:
muestra_estratificada_SMOTE.position.value_counts(normalize=True)

Third Baseman        0.111111
Pitcher              0.111111
Shortstop            0.111111
Outfielder           0.111111
Designated Hitter    0.111111
First Baseman        0.111111
Second Baseman       0.111111
Catcher              0.111111
Infielder            0.111111
Name: position, dtype: float64

Al aplicar el algoritmo SMOTE hemos generado muestras sintéticas con la posición y el salario de los jugadores y la proporción de clases ha quedado modificada, siendo igual el número de jugadores para todas las clases.



## Nivell 3

- Exercici 3

Continua amb el conjunt de dades de tema esportiu i genera una mostra utilitzant el mètode Reservoir sampling.

El algoritmo de reservoir sampling indica lo siguiente:

_A simple and popular but slow algorithm, commonly known as Algorithm R, is due to Alan Waterman.[1]_

_The algorithm works by maintaining a reservoir of size k, which initially contains the first k items of the input._

``` reservorio=[]
k=83 
for i in range(len(players)):
    if i<k:
    reservorio.append(k)
        [...]
```
        
_It then iterates over the remaining items until the input is exhausted. Using one-based array indexing, let i > k be the index of the item currently under consideration._

_The algorithm then generates a random number j between (and including) 1 and i. If j is at most k, then the item is selected and replaces whichever item currently occupies the j-th position in the reservoir. Otherwise, the item is discarded._

``` [...]
    else:
        j=rd.randint(0,i)
        if j<k:
            reservorio[j]=i
```


In effect, for all i, the ith element of the input is chosen to be included in the reservoir with probability k / i. 

Similarly, at each iteration the jth element of the reservoir array is chosen to be replaced with probability $1 / k × k / i = 1 / i$. It can be shown that when the algorithm has finished executing, each item in the input population has equal probability (i.e., $k / n$ ) of being chosen for the reservoir: 

$k / i × ( 1 − 1 / ( i + 1 ) ) × ( 1 − 1 / ( i + 2 ) ) × ( 1 − 1 / ( i + 3 ) ) × . . . × ( 1 − 1 / n ) = k / n$

In [26]:
def muestreo_reservorio(muestra,poblacion):
    reservorio = []
    for i in range(poblacion):
        if i < muestra:
            reservorio.append(i)
        else:
            j = rd.randint(0, i)
            if j < muestra:
                reservorio[j] = i
    return reservorio

In [31]:
rd.seed(1984)
muestra=int(len(players)*0.1)+1
poblacion=len(players)
muestra_reservorio_indices=muestreo_reservorio(muestra,poblacion)
print('Tamaño de la población = ',poblacion)
print('Tamaño muestra = ',muestra)
print('Reservorio = ',muestra_reservorio_indices)


Tamaño de la población =  828
Tamaño muestra =  83
Reservorio =  [463, 314, 396, 518, 416, 453, 272, 797, 466, 325, 718, 502, 815, 622, 320, 15, 631, 191, 109, 599, 192, 655, 735, 533, 633, 621, 231, 394, 160, 536, 357, 612, 507, 583, 350, 670, 36, 432, 209, 303, 485, 342, 182, 755, 44, 460, 275, 114, 410, 307, 638, 373, 430, 319, 269, 55, 407, 392, 783, 785, 317, 61, 465, 397, 489, 431, 266, 98, 68, 386, 261, 71, 72, 154, 408, 462, 745, 531, 195, 213, 149, 88, 318]


In [28]:
muestra_reservorio=players.iloc[muestra_reservorio_indices]

muestra_reservorio

In [30]:
abs(muestra_reservorio.salary.mean()-players.salary.mean())/muestra_reservorio.salary.mean()*100

17.869614503574464

## Resumen

A continuación podemos ver la diferencia entre las medias de las muestras y la media de la población.

In [33]:
metodos={'Muestreo aleatorio simple':muestra_aleatoria,
         'Muestreo sistematico':muestra_sistematica,
         'Muestreo estratificado (PANDAS)':muestra_estratificada_pd,
         'Muestreo estratificado (SKLearn)':muestra_estratificada_SL,
         'Muestreo estratificado con SMOTE':muestra_estratificada_SMOTE,
         'Muestreo reservorio':muestra_reservorio}

In [38]:
tabla=[]
indice=[]
for nombre,metodo in metodos.items():
    valores=[metodo.salary.mean(),
             abs(metodo.salary.mean()-players.salary.mean()),
             abs(metodo.salary.mean()-players.salary.mean())/players.salary.mean()*100]
    indice.append(nombre)
    tabla.append(valores)

round(pd.DataFrame(tabla,
                   columns=['Salario medio', 'Diferencia salario medio población','Pct Diferencia(%)'],
                   index=indice),2)

,Salario medio,Diferencia salario medio población,Pct Diferencia(%)
Muestreo aleatorio simple,3231.14,50.68,1.54
Muestreo sistematico,3350.59,68.76,2.10
Muestreo estratificado (PANDAS),3491.77,209.94,6.40
Muestreo estratificado (SKLearn),3890.10,608.27,18.53
Muestreo estratificado con SMOTE,3352.34,70.52,2.15
Muestreo reservorio,2784.29,497.54,15.16
